# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: Смирнов Александр Львович

## Внимание! Эта версия задания для тех, кто посещает курс в МФТИ.

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.
- [Pytorch](https://pytorch.org/) — инструмент для обучения нейросетей.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

#### Тесты

In [1]:
from tests import TaskTests

task_tests = TaskTests.from_json(path='test_gt.json')

### Вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов. Используйте [модель эмбеддингов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которая была обучена с помощью пакета word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [2]:
import gensim


wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000,
)

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого проверьте три пункта:
1. Используя метод `.most_similar(positive=..., negative=...)`, найти слово, похожее на `woman`, `king` и непохожее на `man`.
2. Используя метод `.doesnt_match(...)`, найти "белую ворону" в списке `['breakfast, 'dinner', 'lunch', 'cereal']`.
3. Используя метод `.most_similar_to_given(word, [...])`, найти наиболее похожее на `music` слово из списка `['water', 'sound', 'backpack', 'mouse']`.

Прокомментируйте полученные результаты: считаете ли вы их верными и почему.

In [3]:
word = "kek"
print(f"is '{word}' exists in embeddings: {word in wv_embeddings}\n")
word = "lord"
print(f"is '{word}' exists in embeddings: {word in wv_embeddings}\n")

positive = ["woman", "king"]
negative = ["man"]
print(f"words like {positive} and opposite to {negative}:\n{wv_embeddings.most_similar(positive=positive, negative=negative)[:3]}\n")

black_sheep = ['breakfast', 'dinner', 'lunch', 'cereal']
print(f"black sheep word in a {black_sheep} list:\n{wv_embeddings.doesnt_match(black_sheep)}\n")

word = "music"
candidates = ['water', 'sound', 'backpack', 'mouse']
print(f"most similar word to '{word}' from {candidates} list:\n{wv_embeddings.most_similar_to_given(word, candidates)}")

is 'kek' exists in embeddings: False

is 'lord' exists in embeddings: True

words like ['woman', 'king'] and opposite to ['man']:
[('queen', 0.7118193507194519), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951)]

black sheep word in a ['breakfast', 'dinner', 'lunch', 'cereal'] list:
cereal

most similar word to 'music' from ['water', 'sound', 'backpack', 'mouse'] list:
sound


Загруженные вектора корректны, так как действительно отражают семантические связи слов.

### Ранжирование вопросов StackOverflow

Давайте посмотрим на данные, которые мы будем использовать в рамках задания. Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (train.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую (валидационную) выборку. Ответьте на следующие вопросы:
1. Сколько пар-дубликатов предоставлено в выборке?
2. Сколько в среднем на каждую пару предоставлено отрицательных примеров?

In [4]:
import tqdm


def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

validation = read_corpus('data/validation.tsv')

In [5]:
import numpy as np

num_samples = len(validation)
amount_of_negatives_per_sample = np.mean(list(map(len, validation))) - 2

In [6]:
task_tests.test_validation_corpus(
    num_samples,
    amount_of_negatives_per_sample
)

### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.



In [7]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
from numpy.linalg import norm
from sklearn.preprocessing import StandardScaler

class Embedder:
    
    def __init__(self, embeddings, dim):
        """
            embeddings: word2vec эмбеддинги
            dim: размерность word2vec эмбеддингов. Нужна для задания нулего вектора для пустых вопросов
        """
        self.embeddings = embeddings
        self.dim = dim
        self.tokenizer = WordPunctTokenizer()
        self.scaler = StandardScaler()
        
    def __call__(self, text, normalize=False):
        """
            Принимает на вход текст и преобразует его в вектор.
            
            text: строка с вопросом
            normalize: при True нужно перед возвращением нормализовать вектор
            
            returns: вектор вопроса
        """
        vector = np.zeros(self.dim)
    
        #text_tokenized = self.tokenizer.tokenize(text.lower())
        text_tokenized = text.split(" ")
        phrase_vectors = [self.embeddings[word] for word in text_tokenized if word in self.embeddings]
        
        if phrase_vectors:
            vector = np.mean(phrase_vectors, axis=0)
        
        if normalize:
            vector = self.scaler.fit_transform(vector.reshape(-1, 1))
        
        return vector.reshape(1, -1)

In [8]:
embedder = Embedder(wv_embeddings, dim=300)

In [9]:
task_tests.test_embedder(embedder)

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

В качестве метрики схожести вопросов будем использовать косинусную близость.

В валидационном датасете для каждой пары вопросов-дубликатов у нас есть случайные отрицательные примеры. Для каждого триплета (вопрос, дубликат, отрицательные примеры) будет ранжировать с помощью нашей модели и косинусной близости дубликат и отрицательные примеры и смотреть на позицию дубликата.

#### Hits@K
Довольно простой и легко интерпретируемой метрикой будет количество корректных попаданий дубликата в top "выдачи" для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### Пример оценок

Пусть $N = 1$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

#### Подсчет метрики Hits@k сразу для нескольких k

Чтобы посчитать метрику для нескольких k, не нужно повторно ранжировать нашей моделью вопросы для одного и того же сэмпла. Достаточно посчитать для сэмпла количество **сложных негативов** - отрицательных примеров, оказавшихся в выдаче выше, чем дубликат. Тогда
$$Hits@k = \begin{cases}
    1, & N < k \\
    0, & иначе
   \end{cases},$$
где **N** - количество сложных негативов.

Реализуйте подсчет Hits@k для произвольного набора значений k и заданной валидационной выборки, используя предложенный шаблон.

In [10]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
        
    
class Scorer:
    
    def __init__(self, k, embedder):
        """
            k: список значений k, для которых нужно посчитать hits@k
            embedder: объект класса Embedder, умеющий преобразовать текст в вектор
        """
        self.k = k
        self.embedder = embedder
        
    def _get_hard_negatives(self, q, pos, negs):
        """
            q: текст вопроса
            pos: текст дубликата
            negs: список из текстов случайных вопросов
            
            result: количество сложных отрицательных примеров, оказавшихся выше положительного
        """
        q_embedding = self.embedder(q)
        pos_embedding = self.embedder(pos)
        negs_embeddings = list(map(self.embedder, negs))
        
        sim_to_pos = cosine_similarity(q_embedding, pos_embedding)
        sims_to_negs = np.array(list(map(lambda neg: cosine_similarity(q_embedding, neg), negs_embeddings)))
        
        result = np.sum(sims_to_negs > sim_to_pos)
        
        return result
    
    def __call__(self, samples, verbose=False):
        """
            samples: список из списков вида [q, pos, neg1, neg2, ...]. Наша валидационная выборка
            verbose: выводить progressbar подсчета метрики с помощью tqdm
            
            result: словарь вида {k: hits@k}
        """
        result = dict(zip(self.k, [0] * len(self.k)))
        
        for sample in tqdm(samples, disable=not verbose):
            hard_negs = self._get_hard_negatives(sample[0], sample[1], sample[2:])
            for k_i in self.k:
                result[k_i] += k_i > hard_negs
            
        for k_i in self.k:
            result[k_i] /= len(samples)
            
        return result

In [ ]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation, verbose=True)

In [ ]:
hits

In [ ]:
task_tests.test_scorer(hits)

### Предобработка текста

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации и всевозможные плохие символы на пробелы;
- Удалить стопслова.
- Удалить слова с длиной меньше трех букв

Реализуйте предобработку текста, используя предложенный шаблон.

In [11]:
import re
import string
from nltk.tokenize import RegexpTokenizer
    
    
class TextPreprocessor:
    
    def __init__(self, characters, min_word_length=0, stopwords=None):
        """
            characters: список плохих символов
            min_word_length: минимальная допустимая длина для слов
            stopwords: множество фоновых слов
        """
        self.characters = characters
        self.min_word_length = min_word_length
        self.stopwords = stopwords
        
        self.pattern = '[' + re.escape(''.join(self.characters)) + ']'
        self.tokenizer = RegexpTokenizer('[a-z0-9+-]+')
        self.characters_table = str.maketrans(' ', ' ', "".join(characters))
        

    def __call__(self, text):
        """
            text: текст для обработки
            
            returns: обработанный текст
        """
        text = text.lower()
       # text = text.translate(self.characters_table)
        
        text = re.sub(self.pattern, ' ', text)
        #text = self.tokenizer.tokenize(text)
        #text = [word for word in text if word not in self.stopwords]
        text = [word for word in text.split() if word not in self.stopwords]
        text = ' '.join(text)
    
        #text = ' '.join([word for word in text.lower().split() if word not in self.stopwords])
        #text = text.translate(self.characters_table)
        
        return text

In [12]:
text_preprocessor = TextPreprocessor(
    characters=('?', '.', '-', ':'),
    stopwords={'not', 'and', 'or'},
    min_word_length=3
)

In [13]:
task_tests.test_text_preprocessor(TextPreprocessor)

Множество фоновых слов можно взять из **nltk** с помощью `nltk.corpus.stopwords.words`, выкидываемые плохие символы и пунктуацию следует подобрать самостоятельно.

Обработайте текст и продемонстрируйте улучшение качества:

In [14]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
    
text_preprocessor = TextPreprocessor(
    characters=string.punctuation,
    min_word_length=1,
    stopwords=nltk.corpus.stopwords.words("english")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/furiousteabag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
validation_preprocessed = [[text_preprocessor(text) for text in row] for row in tqdm(validation)]

100%|██████████| 3760/3760 [01:14<00:00, 50.43it/s]


In [ ]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)
hits = scorer(validation_preprocessed, verbose=True)

hits

Одним из критериев получения полных баллов является значение **hits@500** $\geqslant 0.82$ до предобработки текста и $\geqslant 0.85$ после предобработки.

## Часть 2. Представления для неизвестных слов. (8 баллов)

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм #wh, whe, her, ere, re#

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

### Создание триграммного токенизатора

Для начала, нам нужно:
1. Пройтись по известным в word2vec словам и составить множество триграмм, для которых будем обучать векторы
2. Составить маппинг из триграмм в индексы
3. Реализовать преобразование произвольного слова в список триграмм
4. Реализовать преобразование произвольного слова в список индексов триграмм

Для реализации всех этих пунктов предлагается использовать шаблон, приведенный ниже.

In [16]:
class TrigramTokenizer:
    
    def __init__(self, words):
        """
            Формируем множество всевозможных триграмм, встречающихся в словах из words.
            Делаем маппинг триграмм в индексы.
            
            words: список слов
        """
        self.words = words
        
        self.trigrams = []
        for word in words:
            self.trigrams.append(self._get_trigrams(word))
        
        self.trigrams = set(list(itertools.chain(*self.trigrams)))
        self.trigrams = dict(zip(self.trigrams, range(self.vocab_size)))
            
    @property
    def vocab_size(self):
        """
            returns: колчиество триграмм, для которых мы завели индекс.
        """
        return len(self.trigrams)
    
    @staticmethod
    def _get_trigrams(word):
        """
            word: слово
            
            returns: список триграмм для word
        """
        trigrams = []
        
        word = "#" + word + "#"
        for i in range(1, len(word) - 1):
            trigrams.append(word[i-1:i+2])
        
        return trigrams
            
    def __call__(self, word):
        """
            word: слово
            
            returns: список индексов триграмм для слова word, которые нашлись в маппинге
        """
        trigrams = self._get_trigrams(word)
        trigrams_indexes = [self.trigrams[trigram] for trigram in trigrams if trigram in self.trigrams]
        
        return trigrams_indexes

In [17]:
task_tests.test_trigram_tokenizer(TrigramTokenizer)

Для создания токенизатора используйте обработанный с помощью TextProcessor текст. 

**Важно:** в токенизатор нужно подавать только слова, известные word2vec'у.

In [18]:
import itertools

In [19]:
%%time

sentences_preprocessed = [item for row in validation_preprocessed for item in row]
sentences_words_preprocessed = list(map(lambda sentence: sentence.split(), sentences_preprocessed))
words_preprocessed = set(list(itertools.chain(*sentences_words_preprocessed)))
print(f"Total number of unique words in validation_preprocessed: {len(words_preprocessed)}")

w2v_vocab = list(filter(lambda word: word in wv_embeddings, words_preprocessed))
print(f"Total number of unique words which have embeddings: {len(w2v_vocab)}")

tri_tokenizer = TrigramTokenizer(w2v_vocab)
print(f"Total number of trigrams: {tri_tokenizer.vocab_size}")

Total number of unique words in validation_preprocessed: 17635
Total number of unique words which have embeddings: 7968
Total number of trigrams: 4142
CPU times: user 11.3 s, sys: 553 ms, total: 11.8 s
Wall time: 11.8 s


### Создание датасета с w2v векторами и списками индексов триграмм

Мы будем обучать триграммную модель в нейросетевом фреймворке pytorch. Для этого нам нужно создать свой датасет.

Он должен:
1. Принимать список слов, word2vec и уже созданный триграммный токенизатор.
2. Выдавать пары вида (эмбеддинг для слова из word2vec, список индексов триграмм для этого слова)

Реализовать датасет нужно в шаблоне, приведенном ниже.

In [20]:
from torch.utils.data import Dataset


class TrainTrigramDataset(Dataset):
    
    def __init__(self, vocab, w2v_embeddings, tri_tokenizer):
        """
            Формируем выборку для обучения триграммной модели.
            ЗАРАНЕЕ считаем маппинг в список индексов для всех известных в word2vec слов.
            
            vocab: список слов
            w2v_embeddings: no comments
            tri_tokenizer: токенизатор триграмм
        """
        self.vocab = vocab
        self.w2v_embeddings = w2v_embeddings
        self.tri_tokenizer = tri_tokenizer
        
        
        self.w2tri_index = {}
        for word in self.vocab:
            self.w2tri_index[word] = self.tri_tokenizer(word)
        
                
    def __len__(self):
        """
            returns: возвращает количество слов, вошедших в маппинг (размер словаря)
        """
        return len(self.w2tri_index)
    
    def __getitem__(self, idx):
        """
            returns: w2v эмбеддинг для idx-го слова в датасете, список соответствующих ему триграмм (тензоры)
        """
        word = self.vocab[idx]
        word_vec = self.w2v_embeddings[word]
        word_tri_index = self.w2tri_index[word]
        
        return (word_vec, word_tri_index)
    
ds = TrainTrigramDataset(w2v_vocab, wv_embeddings, tri_tokenizer)

In [21]:
task_tests.test_dataset(ds, w2v_vocab, wv_embeddings, tri_tokenizer)

### Создание DataLoader'а и Collator'а

Нас интересуют в первую очередь четыре параметра при создании DataLoader:
1. Датасет. Реализует интерфейс массива - можно узнать длину и получить элемент с индексом, меньшим длины.
2. batch_size. Задает размера батча (количества сэмплов, идущих одновременно в модель).
3. shuffle. При shuffle == True каждую эпоху при итерировании по даталоадеру мы будем получать сэмплы в произвольном порядке.
4. collate_fn. Этот параметр позволяет задать кастомную логику "склеивания" сэмплов из датасета в батч.

В качестве модели мы будем использовать слой **torch.nn.EmbeddingBag**. Он принимает на вход список индексов и список сдвигов, начинающийся с нуля.

Нужно наш список списков индексов триграмм превратить в соответствующий формат, преобразовать векторы слов и два списка (индексов и сдвигов) в pytorch тензоры (torch.tensor).

Реализуйте следующую функцию:

In [22]:
import torch
from torch.utils.data import DataLoader


def collate_fn(batch):
    """
        batch: список из элементов датасета, e.g. [ds[i] for i in [2, 3, 1, 15]]
        
        returns: w2v эмбеддинги, индексы триграмм, сдвиги для триграмм
    """
    
    w2v_embeddings = []
    index_list = []
    offsets = []

    for vec, indexes in batch:
        w2v_embeddings.append(vec)
        index_list += indexes
        offsets.append(len(indexes))
    
    off = []
    for i, offset in enumerate(offsets):
        if not off:
            off.append(0)
        else:
            off.append(off[-1] + offsets[i-1])
    
    w2v_embeddings = torch.tensor(w2v_embeddings)
    index_list = torch.tensor(index_list)
    offset = torch.tensor(off)
    
    return w2v_embeddings, index_list, offset

In [23]:
task_tests.test_dataloader(ds, collate_fn, embedding_dim=300)

### Создание модели

При создании модели мы обычно наследуемся от **torch.nn.Module** и создаем нужные нам слои как атрибуты объекта нашего класса.

В данном случае предлагается для формирования эмбеддингов использовать **torch.nn.EmbeddingBag**.

Реализуйте предложенный шаблон:

In [24]:
from torch import nn


class TrigramModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество триграмм, для которых обучаются эмбеддинги
            embedding_dim: размерность эмбеддингов триграмм
        """
        super(TrigramModel, self).__init__()
        self.embedding_bag = nn.EmbeddingBag(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
        
    @property
    def embedding_dim(self):
        """
            returns: размерность эмбеддингов
        """
        return self.embedding_bag.embedding_dim
    
    @property
    def num_embeddings(self):
        """
            returns: количество эмбеддингов
        """
        return self.embedding_bag.num_embeddings
    
    def forward(self, trigrams, offsets):
        """
            trigrams: список индексов триграмм (тензор)
            offsets: список сдвигов (тензор)
            
            returns: эмбеддинги слов, составленные из триграмм
        """
        embeddings = self.embedding_bag(trigrams, offsets)
        return embeddings
    
model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size)

In [25]:
task_tests.test_trigram_model(model)

### Создание пайплайна обучения

Далее необходимо совместить все наработки в единый пайплайн обучения, добавив также критерий для оптимизации и оптимизатор. 

Предлагается:

1. В качестве оптимизатора использовать Adam (можно попробовать подобрать learning rate / weight decay)
2. В качестве критерия оптимизации взять nn.MSELoss (можно также закодить лосс самому)
3. Для даталоадера выбрать небольшой батч сайз (32, 64, 128, 256)
4. Десяти эпох должно быть достаточно для хорошего качества

Реализуйте предложенный шаблон.

In [26]:
import time

    
class Trainer:
    
    def __init__(self, model, criterion, optimizer):
        """
            model: триграммная модель
            criterion: функционал ошибки, принимает на вход w2v эмбеддинги и триграммные эмбеддинги
            optimizer: оптимизатор для модели
        """
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def _train_step(self, dataloader):
        """
            Делаем один проход по даталоадеру, с бэкпропом
            
            dataloader: даталоадер с тренировочными данными
            
            returns: лосс
        """
        epoch_loss = 0.0
        
        for words, trigrams, offsets in dataloader:
            self.optimizer.zero_grad()
            
            output = self.model(trigrams, offsets)
            self.loss = self.criterion(output, words)
            self.loss.backward()
            self.optimizer.step()
            
            epoch_loss += self.loss
            
        return epoch_loss / len(dataloader)
        #return epoch_loss

    
    def train(self, dataloader, n_epochs, verbose=False):
        """
            dataloader: тренировочный даталоадер
            n_epochs: количество эпох
            verbose: выводить лосс каждую эпоху или нет
            
            returns: список лоссов
        """
        start = time.time()
        losses = []
        for epoch in range(n_epochs):
            loss = self._train_step(dataloader)
            losses.append(loss)
            if verbose:
                print(f'epoch: {epoch + 1:>2}, loss: {loss:.4f}, time: {time.time() - start:.4f}')
        return losses

In [27]:
BATCH_SIZE = 128
N_EPOCHS = 20
LR = 0.01

model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

trainer = Trainer(model, criterion, optimizer)

dataloader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [28]:
losses = trainer.train(dataloader, N_EPOCHS, verbose=True)

epoch:  1, loss: 0.1759, time: 0.7112
epoch:  2, loss: 0.1271, time: 1.3985
epoch:  3, loss: 0.0968, time: 2.0220
epoch:  4, loss: 0.0776, time: 2.6740
epoch:  5, loss: 0.0641, time: 3.3839
epoch:  6, loss: 0.0541, time: 3.9984
epoch:  7, loss: 0.0470, time: 4.6660
epoch:  8, loss: 0.0413, time: 5.3360
epoch:  9, loss: 0.0367, time: 6.2846
epoch: 10, loss: 0.0334, time: 7.1377
epoch: 11, loss: 0.0304, time: 7.9947
epoch: 12, loss: 0.0280, time: 8.6536
epoch: 13, loss: 0.0261, time: 9.3836
epoch: 14, loss: 0.0245, time: 11.0066
epoch: 15, loss: 0.0232, time: 11.7747
epoch: 16, loss: 0.0220, time: 12.3916
epoch: 17, loss: 0.0210, time: 12.9988
epoch: 18, loss: 0.0202, time: 13.5854
epoch: 19, loss: 0.0195, time: 14.1822
epoch: 20, loss: 0.0189, time: 14.7754


### Получение векторов неизвестных слов. Инференс модели

Теперь, когда мы обучили модель, нам необходимо применить её для всех неизвестных слов, т.е. получить для них эмбеддинги.

Т.к. для этих слов у нас нет word2vec эмбеддингов, то dataset и collator для обучения не подходят для инференса. Необходимо реализовать датасет и коллатор для инференса по следующим шаблонам:

In [29]:
class InferenceTrigramDataset:
    
    def __init__(self, vocab, tri_tokenizer):
        """
            Датасет с неизвестными словами
            
            vocab: список слов
            tri_tokenizer: триграммный токенизатор
        """
        self.initial_vocab = vocab
        self.tri_tokenizer = tri_tokenizer
        
        self.vocab = []
        self.w2tri_index = {}
        for word in self.initial_vocab:
            tokens = self.tri_tokenizer(word)
            if tokens:
                self.w2tri_index[word] = tokens
                self.vocab.append(word)
        
    def __len__(self):
        return len(self.w2tri_index)
    
    def __getitem__(self, idx):
        word = self.vocab[idx]
        word_tri_index = self.w2tri_index[word]
        
        return word_tri_index
    
    
def inference_collate_fn(trigrams):
    """
        trigrams: список списков индексов триграмм
        
        returns: список индексов, список сдвигов триграмм
    """
    index_list = []
    offsets = []

    for indexes in trigrams:
        index_list += indexes
        offsets.append(len(indexes))
    
    off = []
    for i, offset in enumerate(offsets):
        if not off:
            off.append(0)
        else:
            off.append(off[-1] + offsets[i-1])
    
    index_list = torch.tensor(index_list)
    offset = torch.tensor(off)
    
    return index_list, offset

In [30]:
vocab_unk = list(words_preprocessed - set(w2v_vocab))
print(f"Total number of words w/o w2v represetaion: {len(vocab_unk)}")
print(f"Sample words: {vocab_unk[:5]}\n")

inf_ds = InferenceTrigramDataset(vocab_unk, tri_tokenizer)
vocab_unk_exits_trigrams = inf_ds.vocab
print(f"Total number of words w/o w2v represetaion for which exists trigrams: {len(vocab_unk_exits_trigrams)}")
print(f"Sample words that don't have trigrams: {list(set(vocab_unk) - set(vocab_unk_exits_trigrams))[:10]}")

Total number of words w/o w2v represetaion: 9667
Sample words: ['onkeydown', 'ismember', 'namedtuple', 'onnotificationsent', 'dta']

Total number of words w/o w2v represetaion for which exists trigrams: 9172
Sample words that don't have trigrams: ['400', '550m', '42000', '660', '0755', '12c', '23', '08', '⇒', '90º']


In [31]:
inf_dataloader = DataLoader(inf_ds, batch_size=1, shuffle=False, collate_fn=inference_collate_fn)

Теперь у нас есть всё необходимое, чтобы осуществить инференс. Не забудь перед инференсом перевести модель в режим эвала (**model.eval**), а также использовать контекстный менеджер **torch.no_grad**.

После инференса сформируйте словарь из известных в word2vec слов и их эмбеддингов, затем дополните его эмбеддингами для неизвестных слов, полученными после инференса.

#### loss on inference

In [32]:
epoch_loss = 0.0

for words, trigrams, offsets in dataloader:

    output = trainer.model(trigrams, offsets)
    loss = criterion(words, output)

    epoch_loss += loss

print(epoch_loss / len(dataloader))

tensor(0.0175, grad_fn=<DivBackward0>)


#### predicting embeddings

In [33]:
embeddings = []
trainer.model.eval()

with torch.no_grad():
    
    for trigrams, offsets in inf_dataloader:
        embeddings += trainer.model(trigrams, offsets)

#### merging embeddings into single dict

In [34]:
all_words_embeddings = {}

for word in w2v_vocab:
    all_words_embeddings[word] = wv_embeddings[word]
    
for i, word in enumerate(vocab_unk_exits_trigrams):
    all_words_embeddings[word] = embeddings[i].numpy()

In [35]:
print(len(all_words_embeddings))

17140


#### evaluating embeddings

In [36]:
from random import sample

In [37]:
unk_words = sample(vocab_unk_exits_trigrams, 5)
for word in unk_words:
    embedding = all_words_embeddings[word]
    print(f"Most similar wv words to word '{word}': {wv_embeddings.similar_by_vector(embedding, 5)}\n")

Most similar wv words to word 'telephonymanager': [('telephone', 0.5474270582199097), ('telephony', 0.5350211262702942), ('phone', 0.5324443578720093), ('VoIP', 0.5025209784507751), ('VOIP', 0.4857209026813507)]

Most similar wv words to word 'zeroclipboard': [('clipboard', 0.8053747415542603), ('notepad', 0.5378260016441345), ('clipboards', 0.48539432883262634), ('whiteboard', 0.4724689722061157), ('pencil', 0.46339717507362366)]

Most similar wv words to word 'shakelistener': [('shake', 0.423478901386261), ('shaking', 0.3534550368785858), ('%_#F########_3v.jsn', 0.3368300497531891), ('tingles', 0.33558592200279236), ('handshake', 0.32885751128196716)]

Most similar wv words to word 'attributeconverter': [('attribute', 0.5822771787643433), ('attributes', 0.5320720672607422), ('converts', 0.5033000111579895), ('attributed', 0.5025203227996826), ('convert', 0.4853014349937439)]

Most similar wv words to word 'plone': [('plot', 0.5352764129638672), ('plots', 0.4386828541755676), ('plotte

Используя **Scorer** и **Embedder**, получите новые значения метрик для валидации:

In [38]:
embedder = Embedder(all_words_embeddings, dim=300)

scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation_preprocessed, verbose=True)

100%|██████████| 3760/3760 [12:09<00:00,  5.16it/s]


In [39]:
hits

{1: 0.41808510638297874,
 5: 0.5579787234042554,
 10: 0.6066489361702128,
 100: 0.7840425531914894,
 500: 0.9188829787234043,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.89$.

## Дополнительная часть. Обучение векторных представлений для целевой задачи. (до 5 баллов)

Предполагается, что в этой части используются TextPreprocessor, Embedder, Scorer из предыдущих частей.

Для обучения на целевую задачу нам понадобится обучающая выборка. Считайте её с диска, предобработайте текст вопросов

In [40]:
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader

In [41]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [42]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
    
text_preprocessor = TextPreprocessor(
    characters=string.punctuation,
    min_word_length=1,
    stopwords=nltk.corpus.stopwords.words("english")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/furiousteabag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
train = []
for questions in tqdm(read_corpus('data/train.tsv')):
    train.append([text_preprocessor(text) for text in questions])

100%|██████████| 1000000/1000000 [00:44<00:00, 22606.08it/s]


Необходимо создать **токенизатор для текста** - составить словарь и сделать маппинг из слов в индексы.

In [44]:
class TextTokenizer:
    
    def __init__(self, vocab):
        """
            vocab: множество слов, встретившихся в обучающей выборке
        """
        self.vocab = vocab
        self.word2index = dict(zip(vocab, range(len(vocab))))

    @property
    def vocab_size(self):
        """
            returns: количество слов в словаре
        """
        return len(self.word2index)
    
    def __call__(self, text):
        """
            text: текст
            
            returns: список индексов
        """
        indexes = [self.word2index[word] for word in text.split() if word in self.word2index]
        return indexes

Составление словаря и токенизатора

In [45]:
import itertools

In [46]:
%%time

sentences = [item for row in train for item in row]
sentences_words = list(map(lambda sentence: sentence.split(), sentences))
words = set(list(itertools.chain(*sentences_words)))
print(f"Total number of unique words in train dataset: {len(words)}")

Total number of unique words in train dataset: 132383
CPU times: user 5.96 s, sys: 376 ms, total: 6.34 s
Wall time: 6.35 s


In [47]:
text_tokenizer = TextTokenizer(words)

Нам также понадобится **новый датасет для обучения**. Для применения метода NT-Exent нам не нужно "майнить негативы", поэтому датасет надо сформировать как массив из пар-дубликатов.

Так как данные в обучающей выборке содержат множества дубликатов (т.е. все дубликаты сгруппированы в списки), есть несколько способов сформировать итоговый датасет:
1. Оставить из каждого множества дубликатов какие-нибудь случайные два (или просто первые два вопроса)
2. Для первого вопроса в множестве взять все остальные как дубликаты (N вопросов-дубликатов - N-1 пара). Тогда мы увидим каждый вопрос хотя бы один раз при обучении
3. Составить всевозможные уникальные пары-дубликаты из этих множеств (т.е. первый вопрос и все остальные вопросы, второй вопрос и все остальные, кроме первого).

Каждый следующий способ, начиная с первого, раздувает выборку по размеру, но возможно дает прирост к качеству решения задачи.

Реализуйте выбранный вами подход, используя предолженный шаблон:

In [48]:
class QuestionDuplicatesDataset(Dataset):
    
    def __init__(self, question_pairs, tokenizer):
        """
            question_pairs: список из пар вопросов-дубликатов
            tokenizer: объект класса TextTokenizer
        """
        self.question_pairs = question_pairs
        self.tokenizer = tokenizer
        
        self.pairs = []
        for duplicate_questions in self.question_pairs:
            for i in range(len(duplicate_questions)):
                for j in range(i+1, len(duplicate_questions)):
                    self.pairs.append(
                        (self.tokenizer(duplicate_questions[i]),
                         self.tokenizer(duplicate_questions[j]))
                    )
            
    def __len__(self):
        """
            returns: количество пар-дубликатов
        """
        return len(self.pairs)
    
    def __getitem__(self, idx):
        """
            returns: (вопрос, дубликат), idx-ю пару в датасете
        """
        return self.pairs[idx]

Также нужно подготовить **даталоадер** (а именно - коллатор для даталоадера) по аналогии со второй частью задания.

In [49]:
import torch

In [50]:
def get_ids_and_offsets(questions):
    """
        questions: список из токенизированных вопросов
        
        returns: (ids, offsets), где ids - вытянутый список индексов слов в вопросах из батча, offsets - сдвиги
    """
    ids = []
    offsets = []

    for indexes in questions:
        ids += indexes
        offsets.append(len(indexes))
    
    off = []
    for i, offset in enumerate(offsets):
        if not off:
            off.append(0)
        else:
            off.append(off[-1] + offsets[i-1])
    
    return (torch.tensor(ids), torch.tensor(off))


def collate_fn(batch):
    """
        batch: список из пар токенизированных вопросов-дубликатов [(question, duplicate), ...]
        
        returns: (question_ids, question_offsets), (duplicate_ids, duplicate_offsets)
    """
    questions, duplicates = map(list, zip(*batch))
    questions, duplicates = get_ids_and_offsets(questions), get_ids_and_offsets(duplicates)
    
    return questions, duplicates

Поделите выборку на трейн и валидацию, используя train_test_split, затем **создайте датасеты и даталоадеры** для обучения и валидации. Сколько пар-дубликатов получилось в датасете для обучения?

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
%%time

train_split, validation_split = train_test_split(train, test_size=0.2)
print(f"Total number of rows in initial train dataset: {len(train)}")
print(f"Total number of rows in train split: {len(train_split)}")
print(f"Total number of rows in validation split: {len(validation_split)}\n")

train_dataset = QuestionDuplicatesDataset(train_split, text_tokenizer)
validation_dataset = QuestionDuplicatesDataset(validation_split, text_tokenizer)
print(f"Number of pairs in train dataset: {len(train_dataset)}")
print(f"Number of pairs in validation dataset: {len(validation_dataset)}")

Total number of rows in initial train dataset: 1000000
Total number of rows in train split: 800000
Total number of rows in validation split: 200000

Number of pairs in train dataset: 1297623
Number of pairs in validation dataset: 322550
CPU times: user 13 s, sys: 180 ms, total: 13.2 s
Wall time: 13.2 s


С помощью предложенного шаблона **задайте модель** для преобразования вопросов в векторы.

In [53]:
from torch import nn

class DssmLikeModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество слов, для которых обучаем эмбеддинги
            embedding_dim: размерность эмбеддинга
        """
        super().__init__()
        self.embedding_bag = nn.EmbeddingBag(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
        
    @property
    def embedding_dim(self):
        """
            returns: размерность эмбеддингов
        """
        return self.embedding_bag.embedding_dim
    
    @property
    def num_embeddings(self):
        """
            returns: количество эмбеддингов
        """
        return self.embedding_bag.num_embeddings
        
    def forward(self, ids, offsets):
        """
            ids: вытянутая посл-ть индексов слов вопросов, попавших в батч
            offsets: сдвиги для вопросов, попавших в батч
            
            returns: векторы вопросов
        """
        embeddings = self.embedding_bag(ids, offsets)
        return embeddings

Создание модели

**Критерий оптимизации** для NTExentLoss выглядит как:

$$\mathcal{L}(Q, D) = -0.5 \log diag(softmax(QD^T / \alpha)) - 0.5 \log diag(softmax(DQ^T / \alpha)),$$

где:
* $Q \in \mathbb{R}^{b \times d}$ - эмбеддинги вопросов, 
* $D \in \mathbb{R}^{b \times d}$ - эмбеддинги соответствующих вопросам дубликатов,
* $b$ - количество пар (вопрос, дубликат), $d$ - размерность эмбеддингов, $\alpha$ - гиперпараметр лосса. 
* Softmax берется по рядам
* Матрицы $Q, D$ содержат нормированные эмбеддинги, т.е. считается именно косинус.

In [54]:
class NTExentLoss(nn.Module):
    
    def __init__(self, alpha=1., eps=1e-8):
        """
            alpha: коэффициент, на который мы делим скоры перед софтмаксом
            eps: ||v|| = max(eps, ||v||)
        """
        super().__init__()
        self.alpha = alpha
        self.eps = eps
        
        self.softmax = nn.Softmax(dim=0)
        
    def _normalize(self, embeddings):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            
            returns: матрица такого же размера, но с нормироваными векторами
        """
        return nn.functional.normalize(embeddings, dim=0, eps=self.eps)
    
    def forward(self, embeddings, positives):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            positives: матрица такого же размера, с позитивами для векторов из матрицы embeddings
            
            returns: NT-Exent loss
        """
        first = -0.5 * torch.log(
            torch.diag(
                self.softmax(
                    embeddings @ positives.T / self.alpha)) + self.eps)
        
        second = 0.5 * torch.log(
            torch.diag(
                self.softmax(
                    positives @ embeddings.T / self.alpha)) + self.eps)
        
        return torch.mean(first - second)

**Создайте пайплайн** для обучения и валидации, используя предложенный шаблон.

Залогируйте с помощью **torch.utils.tensorboard.SummaryWriter** две величины:
1. Лосс для каждого батча
2. Лосс на валидации для каждой эпохи

In [55]:
import time
import shutil
import os
import torch

from torch.utils.tensorboard import SummaryWriter
    
    
class Trainer:
    
    def __init__(
            self, 
            model, 
            optimizer, 
            criterion, 
            logdir=None, 
            device=None
    ):
        """
            model: объект класса DssmModel
            optimizer: оптимизатор
            criterion: критерий оптимизации
            logdir: директория, в которую SummaryWriter должен писать логи
            device: девайс (cpu или cuda), на котором надо производить вычисления
        """
        self.device = device
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.criterion = criterion.to(self.device)
        self.logdir = logdir
        self._writer = SummaryWriter(log_dir=logdir)
    
    def _calculate_loss(self, batch):
        """
            batch: батч из индексов и сдвигов для вопросов и их дубликатов
            
            returns: посчитанный для батча лосс
        """
        questions, duplicates = batch

        q_indexes, q_offsets = questions
        q_indexes = q_indexes.to(self.device)
        q_offsets = q_offsets.to(self.device)
        
        d_indexes, d_offsets = duplicates
        d_indexes = d_indexes.to(self.device)
        d_offsets = d_offsets.to(self.device)
        
        embeddings = self.model(q_indexes, q_offsets)
        positives = self.model(d_indexes, d_offsets)

        loss = self.criterion(embeddings, positives)
        return loss
    
    def _train_step(self, dataloader):
        """
            dataloader: даталоадер для обучения
            
            returns: лосс на датасете для обучения
        """
        self.model.train()
        epoch_loss = 0.0
        
        for batch in dataloader:
            self.optimizer.zero_grad()
            
            loss = self._calculate_loss(batch)
            loss.backward()
            self.optimizer.step()
            
            epoch_loss += loss
            
        return epoch_loss / len(dataloader)
    
    def _eval_step(self, dataloader):
        """
            dataloader: даталоадер для валидации
            
            returns: лосс на валидации
        """
        self.model.eval()
        
        epoch_loss = 0.0
        
        with torch.no_grad():
            for batch in dataloader:
                loss = self._calculate_loss(batch)
                epoch_loss += loss
            
        return epoch_loss / len(dataloader)
    
    def train(self, dataloaders, n_epochs, verbose=False):
        """
            dataloaders: словарь вида {'train': train_dataloader, 'eval': eval_dataloader}
            n_epochs: количество эпох обучения
            verbose: нужно ли выводить каждую эпоху информацию про лоссы
        """
        start = time.time()
        self._n_epoch = 1
        for epoch in range(n_epochs):
            train_loss = self._train_step(dataloaders['train'])
            
            eval_loss = self._eval_step(dataloaders['eval'])
            if self._writer is not None:
                self._writer.add_scalar('eval/loss', eval_loss, global_step=self._n_epoch)
                
            if verbose:
                print(
                    'epoch: {:>2}, train loss: {:.4f}, eval loss: {:.4f}, time: {:.4f}' \
                        .format(epoch + 1, train_loss, eval_loss, time.time() - start)
                )
                    
            self._n_epoch += 1

Предлагается использовать для оптимизации Адам и обучать модель 10-60 эпох.

Для этой части задания GPU даёт существенное ускорение при обучении, поэтому стоит по возможности делать обучение с большим batch size'ом и на GPU.

In [56]:
#BATCH_SIZE = 4
BATCH_SIZE = 4096
#BATCH_SIZE = 8192
#BATCH_SIZE = 16384

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [57]:
dataloaders = {
    "train": train_dataloader,
    "eval": validation_dataloader
}

In [58]:
EMBEDDING_DIM = 300
N_EPOCHS = 50
LR = 0.001
WEIGHT_DECAY = 1e-6
logdir = "./logs/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = DssmLikeModel(text_tokenizer.vocab_size, EMBEDDING_DIM)
criterion = NTExentLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

cuda


In [59]:
trainer = Trainer(model, optimizer, criterion, logdir, device)

In [60]:
trainer.train(dataloaders, N_EPOCHS, verbose=True)

epoch:  1, train loss: 11.9334, eval loss: 12.3762, time: 46.3989
epoch:  2, train loss: 10.2640, eval loss: 11.1830, time: 89.9719
epoch:  3, train loss: 8.7858, eval loss: 10.0904, time: 133.4530
epoch:  4, train loss: 7.5474, eval loss: 9.1649, time: 176.7867
epoch:  5, train loss: 6.6142, eval loss: 8.4721, time: 220.1048
epoch:  6, train loss: 5.9727, eval loss: 7.9530, time: 263.4948
epoch:  7, train loss: 5.5196, eval loss: 7.5828, time: 306.7961
epoch:  8, train loss: 5.1931, eval loss: 7.2998, time: 350.0799
epoch:  9, train loss: 4.9378, eval loss: 7.0731, time: 396.2152
epoch: 10, train loss: 4.7288, eval loss: 6.8930, time: 439.4272
epoch: 11, train loss: 4.5568, eval loss: 6.7465, time: 482.6341
epoch: 12, train loss: 4.4134, eval loss: 6.6253, time: 527.0500
epoch: 13, train loss: 4.2901, eval loss: 6.5240, time: 570.5181
epoch: 14, train loss: 4.1770, eval loss: 6.4400, time: 614.0843
epoch: 15, train loss: 4.0859, eval loss: 6.3685, time: 657.3347
epoch: 16, train loss:

Осталось достать из модели обученные под задачу векторы слов, составить маппинг слов в векторы, создать **Embedder** и **Scorer** и провалидировать качество на нашей исходной валидации, которой мы пользовались в первых двух частях.

Чтобы достать из модели веса, можно использовать `model._embeddings.weight.cpu().detach().numpy()`

In [61]:
weights = trainer.model.embedding_bag.weight.cpu().detach().numpy()

In [68]:
embeds = dict(zip(text_tokenizer.vocab, weights))

In [75]:
embedder = Embedder(embeds, dim=300)

scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation_preprocessed, verbose=True)

100%|██████████| 3760/3760 [12:21<00:00,  5.07it/s]


In [76]:
hits

{1: 0.5446808510638298,
 5: 0.6976063829787233,
 10: 0.7473404255319149,
 100: 0.8962765957446809,
 500: 0.9787234042553191,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.98$.